In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
%matplotlib inline

In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score

documents = documents = ['IO','Throughput','of','disk'],
['Average','Queue','Length','of','each','disk'],
['Average','Request','Latency','of','each','disk'],
['Average','Request','Size','of','each','disk'],
['Number','of','Packets','Received'],
['Number','of','Packets','Sent'],
['Current','Receive','Rate','Trend'],
['Current','Send','Rate','Trend'],
['Packet','Drop','Trend'],
['Error','Packet','Trend'],
['Port','Usage']                        ]

vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(documents)

true_k = 3
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(true_k):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),
    print

print("\n")
print("Prediction")

Y = vectorizer.transform(["network Packets congestion"])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(["network Packets congestion"])
prediction = model.predict(Y)
print(prediction)

Top terms per cluster:
Cluster 0:
 disk
 average
 request
 usage
 port
 throughput
 io
 size
 latency
 length
Cluster 1:
 trend
 packet
 current
 rate
 drop
 error
 send
 receive
 usage
 number
Cluster 2:
 number
 packets
 sent
 received
 usage
 packet
 current
 disk
 drop
 error


Prediction
[2]
[2]


In [20]:
from gensim.models import word2vec
import nltk as nltk
# tokenize sentences in corpus
wpt = nltk.WordPunctTokenizer()
tokenized_corpus = [wpt.tokenize(document) for document in documents]

# Set values for various parameters
feature_size = 100    # Word vector dimensionality  
window_context = 30          # Context window size                                                                                    
min_word_count = 1   # Minimum word count                        
sample = 1e-3   # Downsample setting for frequent words

w2v_model = word2vec.Word2Vec(tokenized_corpus, size=feature_size, 
                          window=window_context, min_count=min_word_count,
                          sample=sample, iter=50)

# view similar words based on gensim's model
similar_words = {search_term: [item[0] for item in w2v_model.wv.most_similar([search_term], topn=5)]
                  for search_term in ['disk']}
similar_words

{'disk': ['Length', 'Sent', 'Packet', 'Number', 'Average']}

In [22]:
comment = [['IO','Throughput','of','disk'],
['Average','Queue','Length','of','each','disk'],
['Average','Request','Latency','of','each','disk'],
['Average','Request','Size','of','each','disk'],
['Number','of','Packets','Received'],
['Number','of','Packets','Sent'],
['Current','Receive','Rate','Trend'],
['Current','Send','Rate','Trend'],
['Packet','Drop','Trend'],
['Error','Packet','Trend'],
['Port','Usage']]

In [39]:
comment=['IO Throughput of disk',
'Average Queue Length of each disk',
'Average Request Latency of each disk',
'Average Request Size of each disk',
'Number of Packets Received',
'Number of Packets Sent',
'Current Receive Rate Trend',
'Current Send Rate Trend',
'Packet Drop Trend',
'Error Packet Trend',
'Port Usage']

In [38]:
for i in comment:
    print(i)

IO Throughput of disk
Average Queue Length of each disk
Average Request Latency of each disk
Average Request Size of each disk
Number of Packets Received
Number of Packets Sent
Current Receive Rate Trend
Current Send Rate Trend
Packet Drop Trend
Error Packet Trend
Port Usage


In [ ]:
bin/pip3 install spacy && bin/python3 -m spacy download en

In [40]:

import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

In [41]:
import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)


[nltk_data] Downloading package wordnet to /home/system/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [42]:
nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

[nltk_data] Downloading package stopwords to /home/system/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [85]:
def prepare_text_for_lda(text):
    tokens = tokenize(text)
    tokens = [token for token in tokens if len(token) > 3]
    
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

In [124]:
import random
text_data = []
with open('dataset3.csv') as f:
    for line in f:
        tokens = prepare_text_for_lda(line)
        if random.random() > .30:
            print(tokens)
            text_data.append(tokens)

['disk', 'queue', 'length']
['disk', 'request', 'latency']
['curent', 'packet', 'send', 'rate', 'trend']
['error', 'packet', 'trend']


In [125]:
from gensim import corpora
dictionary = corpora.Dictionary(text_data)
corpus = [dictionary.doc2bow(text) for text in text_data]
import pickle
pickle.dump(corpus, open('corpus.pkl', 'wb'))
dictionary.save('dictionary.gensim')

In [126]:
import gensim
NUM_TOPICS = 2
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)
ldamodel.save('model5.gensim')
topics = ldamodel.print_topics(num_words=4)
for topic in topics:
    print(topic)

(0, '0.217*"disk" + 0.130*"latency" + 0.130*"length" + 0.130*"queue"')
(1, '0.185*"trend" + 0.185*"packet" + 0.111*"rate" + 0.111*"send"')


In [129]:
new_doc = 'Network Congestion Rate'
new_doc = prepare_text_for_lda(new_doc)
new_doc_bow = dictionary.doc2bow(new_doc)
print(new_doc_bow)
print(ldamodel.get_document_topics(new_doc_bow))

[(7, 1)]
[(0, 0.26255834), (1, 0.7374416)]
